In [30]:
import pandas as pd
import re
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/carlosruiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df2

In [20]:
df = pd.read_csv('Vault_diversity_reviews - Vault_diversity_reviews.csv',index_col='Unnamed: 0')
df

,company,company_rating,title,content,author,year
0,Kenway Consulting,3.5,diversity_review,As someone who has visibility into the financi...,Executive in Data & AnalyticsChicago,['2020']
1,Kenway Consulting,3.5,diversity_review,We have a guiding principle to avoid collectiv...,Partner in Data & AnalyticsScottsdale,['2020']
2,Kenway Consulting,3.5,diversity_review,"We hire based on skill set, your potential to ...",Executive in Data & AnalyticsChicago,['2020']
3,Kenway Consulting,3.5,diversity_review,Our CEO has an equal representation of direct ...,Executive in Business OperationsChicago,['2020']
4,Kenway Consulting,3.5,diversity_review,Diversity for women is effective. Diversity fo...,Executive in CybersecurityChicago,['2020']
...,...,...,...,...,...,...
1212,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['The firm accepts people of all race, gender ...",Mid-levelPalo Alto,['2018']
1213,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['Best: The gender ratio is fairly 1:1 and eve...,San Jose,['2018']
1214,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['new class of ~50+, noticeably male-dominant'...",Palo Alto,['2018']
1215,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['We have very serious issues with diversity a...,Palo Alto,['2018']


In [21]:
def clean_text(text):
    '''
    Uses regex for removing punctuation and special characters.
    '''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub(r'^?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    return text

re_clean = lambda x: clean_text(x)

In [23]:
df['clean_review'] = pd.DataFrame(df.content.apply(re_clean))
df

,company,company_rating,title,content,author,year,clean_review
0,Kenway Consulting,3.5,diversity_review,As someone who has visibility into the financi...,Executive in Data & AnalyticsChicago,['2020'],as someone who has visibility into the financi...
1,Kenway Consulting,3.5,diversity_review,We have a guiding principle to avoid collectiv...,Partner in Data & AnalyticsScottsdale,['2020'],we have a guiding principle to avoid collectiv...
2,Kenway Consulting,3.5,diversity_review,"We hire based on skill set, your potential to ...",Executive in Data & AnalyticsChicago,['2020'],we hire based on skill set your potential to ...
3,Kenway Consulting,3.5,diversity_review,Our CEO has an equal representation of direct ...,Executive in Business OperationsChicago,['2020'],our ceo has an equal representation of direct ...
4,Kenway Consulting,3.5,diversity_review,Diversity for women is effective. Diversity fo...,Executive in CybersecurityChicago,['2020'],diversity for women is effective diversity fo...
...,...,...,...,...,...,...,...
1212,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['The firm accepts people of all race, gender ...",Mid-levelPalo Alto,['2018'],the firm accepts people of all race gender ...
1213,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['Best: The gender ratio is fairly 1:1 and eve...,San Jose,['2018'],best the gender ratio is fairly and eve...
1214,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,"['new class of ~50+, noticeably male-dominant'...",Palo Alto,['2018'],new class of noticeably male dominant ...
1215,Frank Rimerman + Co. LLP\nRelated Internships:...,4.5,diversity_review,['We have very serious issues with diversity a...,Palo Alto,['2018'],we have very serious issues with diversity a...


In [31]:
stop_words = set(stopwords.words('english'))


In [32]:
df.company_rating.describe()

count    1217.000000
mean        4.084634
std         0.246326
min         3.500000
25%         4.000000
50%         4.000000
75%         4.000000
max         4.500000
Name: company_rating, dtype: float64